# 0. Dataset
There are 12 types of object class for World Robot Summit(WRS), we used 250 images.    
Dataset is collected by Rasmus, https://github.com/RasmusHaugaard/wrs-data-collection    

We split 200 for training and 50 for validation, each image pretty always contains 12 objects.    
For EfficientDet, we go with COCO format.

# 1. Environment set up
We use the EfficientDet tensorflow official version by Google.
https://github.com/google/automl.git

You also could read the tutorial notebook in the github repo.

In [ ]:
!git clone https://github.com/google/automl.git
!pip install -r requirements.txt

# 2. Train
To train our custom model, we correctly need to prepare some files.



In [ ]:
#(effdet) ~/djplace/automl/efficientdet$ pwd
#/home/ubuntu/djplace/automl/efficientdet
!tree -d
'''
├── wrs
│   ├── annotations
│   ├── train2017
│   └── val2017
'''



annotations/    
instances_train2017.json  instances_val2017.json

train2017/    
200 images for training   

val2017/    
50 images for validation    

In [1]:
# train
!PYTHONPATH=".:$PYTHONPATH" python dataset/create_coco_tfrecord.py --image_dir=wrs/train2017/ --object_annotations_file=wrs/annotations/instances_train2017.json --output_file_prefix=wrs/train

# validation
!PYTHONPATH=".:$PYTHONPATH" python dataset/create_coco_tfrecord.py --image_dir=wrs/val2017/ --object_annotations_file=wrs/annotations/instances_val2017.json --output_file_prefix=wrs/valid

python: can't open file 'dataset/create_coco_tfrecord.py': [Errno 2] No such file or directory
python: can't open file 'dataset/create_coco_tfrecord.py': [Errno 2] No such file or directory


In [ ]:
!mkdir wrs/tfrecords
!mv wrs/*.records wrs/tfrecords/

In [ ]:
!tree -d
'''
├── wrs
│   ├── annotations
│   ├── test
│   ├── tfrecords
│   ├── train2017
│   └── val2017
'''

In [ ]:
# hyper-parameter yaml file for your custom model
!vi wrs.yaml
'''
use_keras_model: False
num_classes: 13
label_map: {1: obj1, 2: obj2, 3: obj3, 4: obj4, 5: obj5, 6: obj6, 7: obj7, 8: obj8, 9: obj9, 10: obj10, 11: obj11, 12: obj12}
moving_average_decay: 0
var_freeze_expr: '(efficientnet|fpn_cells)'
mixed_precision: True
'''

In [ ]:
# pretrained weight on the COCO dataset    
# also can download another models from d0 ~ d7.    
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco2/efficientdet-d4.tar.gz
!tar xzf efficientdet-d4.tar.gz

In [ ]:
!python main.py --mode=train_and_eval --training_file_pattern=wrs/tfrecords/train*.tfrecord --validation_file_pattern=wrs/tfrecords/valid*.tfrecord --model_name=efficientdet-d4 --model_dir=efficientdet-d4-finetune --ckpt=efficientdet-d4 --train_batch_size=16 --num_examples_per_epoch=200 --num_epochs=100 --hparams=wrs.yaml

# Using specific GPU, change the GPU device number
# !CUDA_VISIBLE_DEVICES=1 python main.py --mode=train_and_eval --training_file_pattern=wrs/tfrecords/train*.tfrecord --validation_file_pattern=wrs/tfrecords/valid*.tfrecord --model_name=efficientdet-d1 --model_dir=efficientdet-d1-finetune --ckpt=efficientdet-d1 --train_batch_size=4 --num_examples_per_epoch=40 --num_epochs=1000 --hparams=wrs.yaml

# 3. Inference
Our images are RGB-A 4 channel images, therefore we need to transform it from 4 channel to RGB 3 channel.    

In [ ]:
!python wrs_RGBA2RGB.py

# inference
!python model_inspect.py --runmode=saved_model --model_name=efficientdet-d4 --saved_model_dir=tmp/saved_model/ --ckpt_path=efficientdet-d4-finetune/ --hparams=wrs.yaml --batch_size=1
!python model_inspect.py --runmode=saved_model_infer --model_name=efficientdet-d4 --ckpt_path=wrs_efficientdet/efficientdet-d4-finetune/ --saved_model_dir=tmp/saved_model/efficientdet-d4_frozen.pb --input_image=./wrs/test/*.png --output_image_dir=tmp/img_output --hparams=wrs.yaml